# Guide pour l'Appel à une API Hugging Face pour la Segmentation d'Images

Bienvenue ! Ce notebook a pour but de vous guider pas à pas dans l'utilisation de l'API d'inférence de Hugging Face pour effectuer de la segmentation d'images. La segmentation d'images consiste à attribuer une étiquette (comme "cheveux", "vêtement", "arrière-plan") à chaque pixel d'une image.

Nous allons :
1. Comprendre ce qu'est une API et comment s'y connecter.
2. Envoyer une image à un modèle de segmentation hébergé sur Hugging Face.
3. Récupérer et interpréter les résultats.
4. Visualiser les masques de segmentation.
5. Étendre cela pour traiter plusieurs images.

## 1. Configuration Initiale et Importations

Commençons par importer les bibliothèques Python nécessaires. Nous aurons besoin de :
- `os` pour interagir avec le système de fichiers (lister les images).
- `requests` pour effectuer des requêtes HTTP vers l'API.
- `PIL (Pillow)` pour manipuler les images.
- `matplotlib.pyplot` pour afficher les images et les masques.
- `numpy` pour la manipulation des tableaux (les images sont des tableaux de pixels).
- `tqdm.notebook` pour afficher une barre de progression (utile pour plusieurs images).
- `base64` et `io` pour décoder les masques renvoyés par l'API.

In [ ]:
import os
import requests
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
import base64
import io
import time

# Pause (seconds) entre appels API pour être courtois et éviter le rate limiting
PAUSE_BETWEEN_CALLS = 1.5


### Variables de Configuration

Nous devons définir quelques variables :
- `image_dir`: Le chemin vers le dossier contenant vos images. **Assurez-vous de modifier ce chemin si nécessaire.**
- `max_images`: Le nombre maximum d'images à traiter (pour ne pas surcharger l'API ou attendre trop longtemps).
- `api_token`: Votre jeton d'API Hugging Face. **IMPORTANT : Gardez ce jeton secret !**

**Comment obtenir un token API Hugging Face ?**
1. Créez un compte sur [huggingface.co](https://huggingface.co/).
2. Allez dans votre profil -> Settings -> Access Tokens.
3. Créez un nouveau token (par exemple, avec le rôle "read").
4. Copiez ce token ici.

In [ ]:
# Configuration pour le projet Fashion Trend Intelligence
import os
from dotenv import load_dotenv

# Charger les variables d'environnement depuis le fichier .env
load_dotenv()

# Utiliser la variable d'environnement pour le token API (plus sécurisé)
api_token = os.getenv("HUGGINGFACE_API_KEY")

# Mise à jour des chemins pour la structure de votre projet
image_dir = "../assets/images"  # Images stockées dans le dossier assets
max_images = 1  # Commençons avec peu d'images

# Créer le dossier assets s'il n'existe pas
if not os.path.exists(image_dir):
    os.makedirs(image_dir)
    print(f"Dossier '{image_dir}' créé. Veuillez y ajouter des images (.jpg ou .png).")
else:
    print(f"Dossier '{image_dir}' existant.")

if not api_token or api_token == "your_huggingface_api_key_here":
    print("\n⚠️  ATTENTION : Veuillez configurer votre HUGGINGFACE_API_KEY dans le fichier .env")
    print("📖 Obtenez votre token depuis : https://huggingface.co/settings/tokens")
else:
    print(f"✅ Token API chargé avec succès (se terminant par : ...{api_token[-8:]})")

## 2. Comprendre l'API d'Inférence Hugging Face

L'API d'inférence permet d'utiliser des modèles hébergés sur Hugging Face sans avoir à les télécharger ou à gérer l'infrastructure.

- **Modèle utilisé** : Nous allons utiliser le modèle `sayeed99/segformer_b3_clothes`, spécialisé dans la segmentation de vêtements et de parties du corps.
- **URL de l'API** : L'URL pour un modèle est généralement `https://api-inference.huggingface.co/models/NOM_DU_MODELE`.
- **Headers (En-têtes)** : Pour s'authentifier et spécifier le type de contenu, nous envoyons des en-têtes avec notre requête.
    - `Authorization`: Contient notre token API (précédé de `Bearer `).
    - `Content-Type`: Indique que nous envoyons une image au format JPEG (ou PNG selon le cas).

In [ ]:
API_URL = "https://router.huggingface.co/hf-inference/models/sayeed99/segformer_b3_clothes"  # Remplacez ... par le bon endpoint.
headers = {
    "Authorization": f"Bearer {api_token}"
    # Le "Content-Type" sera ajouté dynamiquement lors de l'envoi de l'image
}

# Import des fonctions utilitaires
from oc_p2_fashion_trend_intelligence.utils.image_utils import (
    validate_single_image,
    get_image_dimensions,
    decode_base64_mask,
    create_masks,
    get_content_type
)

## 3. Configuration et Validation des Images

Cette section regroupe **toutes les constantes configurables** du notebook pour faciliter la personnalisation :

**📝 Configuration modifiable :**
- `ALLOWED_EXTENSIONS`: Extensions d'images supportées 
- `CLASS_MAPPING`: Classes spécifiques au modèle `segformer_b3_clothes`

**🔍 Validation automatique :**
- Listing des images dans le dossier défini
- Validation du format et de la taille des images

**🛠️ Fonctions utilitaires (importées depuis `image_utils`) :**
- `get_image_dimensions`: Récupérer les dimensions d'une image
- `decode_base64_mask`: Décoder un masque base64 
- `create_masks`: Combiner les masques de segmentation

💡 **Astuce** : Modifiez `ALLOWED_EXTENSIONS` si vous voulez supporter d'autres formats d'image !

In [ ]:
# =============================================================================
# CONFIGURATION DU NOTEBOOK - MODIFIEZ SELON VOS BESOINS
# =============================================================================

# Extensions d'images supportées
ALLOWED_EXTENSIONS = (".png", ".jpg", ".jpeg", ".bmp", ".webp")

# Configuration spécifique au modèle segformer_b3_clothes
CLASS_MAPPING = {
    "Background": 0,
    "Hat": 1,
    "Hair": 2,
    "Sunglasses": 3,
    "Upper-clothes": 4,
    "Skirt": 5,
    "Pants": 6,
    "Dress": 7,
    "Belt": 8,
    "Left-shoe": 9,
    "Right-shoe": 10,
    "Face": 11,
    "Left-leg": 12,
    "Right-leg": 13,
    "Left-arm": 14,
    "Right-arm": 15,
    "Bag": 16,
    "Scarf": 17
}

# =============================================================================
# LISTING ET VALIDATION DES IMAGES
# =============================================================================

# Lister les chemins des images à traiter
image_paths = [
    os.path.join(image_dir, filename) 
    for filename in os.listdir(image_dir) 
    if filename.endswith(ALLOWED_EXTENSIONS)
]

# Valider les images (format, taille, etc.)
valid_images = [img_path for img_path in image_paths if validate_single_image(img_path, max_size=(1024, 1024))]

if not valid_images:
    print(f"Aucune image valide trouvée dans '{image_dir}'. Veuillez y ajouter des images.")
else:
    print(f"{len(valid_images)} image(s) valide(s) sur {len(image_paths)} à traiter.")
    
# Les fonctions utilitaires (get_image_dimensions, decode_base64_mask, create_masks) 
# sont maintenant disponibles via l'import depuis image_utils

## 4. Segmentation d'une Seule Image

Avant de traiter toutes les images, concentrons-nous sur une seule pour bien comprendre le processus.

Étapes :
1.  Choisir une image.
2.  Ouvrir l'image en mode binaire (`"rb"`) et lire son contenu (`data`).
3.  Déterminer le `Content-Type` (par exemple, `"image/jpeg"` ou `"image/png"`).
4.  Envoyer la requête POST à l'API avec `requests.post()` en passant l'URL, les headers et les données.
5.  Vérifier le code de statut de la réponse. Une erreur sera levée si le code n'est pas 2xx (succès) grâce à `response.raise_for_status()`.
6.  Convertir la réponse JSON en un dictionnaire Python avec `response.json()`.
7.  Utiliser nos fonctions `get_image_dimensions` et `create_masks` pour obtenir le masque final.
8.  Afficher l'image originale et le masque segmenté.

In [ ]:
if image_paths:
    single_image_path = image_paths[0]  # Prenons la première image de notre liste
    print(f"Traitement de l'image : {single_image_path}")

    try:
        with open(single_image_path, "rb") as image_file:
            image_data = image_file.read()

        # Déterminer dynamiquement le content-type pour tous les formats supportés
        headers["Content-Type"] = get_content_type(single_image_path, ALLOWED_EXTENSIONS)

        # Maintenant, utilisé l'API huggingface
        # ainsi que les fonctions données plus haut pour ségmenter vos images.
        response = requests.post(API_URL, headers=headers, data=image_data)

        if response.status_code == 200:
            print("✅ Segmentation réussie !")
            # Traitez la réponse ici (par exemple, enregistrez le résultat)
            segmentation_result = response.json()
            
            image_size = get_image_dimensions(single_image_path)
            print(f"📏 Dimensions de l'image : {image_size}")
            mask = create_masks(segmentation_result, image_size[0], image_size[1], CLASS_MAPPING)

            # Préparation de la visualisation
            original = Image.open(single_image_path).convert("RGB")

            # Palette simple: générer des couleurs reproductibles pour chaque classe
            rng = np.random.default_rng(42)
            num_classes = max(CLASS_MAPPING.values()) + 1
            palette = np.zeros((num_classes, 3), dtype=np.uint8)
            palette[0] = [0, 0, 0]  # Background
            for name, cid in CLASS_MAPPING.items():
                if cid != 0:
                    palette[cid] = rng.integers(0, 255, size=3)

            color_mask = palette[mask]

            # Histogramme des classes présentes
            unique, counts = np.unique(mask, return_counts=True)
            class_counts = dict(zip(unique.tolist(), counts.tolist()))

            # Affichage (2 colonnes: Original, Masque colorisé)
            fig, axes = plt.subplots(1, 2, figsize=(10, 5))
            axes[0].imshow(original)
            axes[0].set_title("Image Originale")
            axes[0].axis('off')

            axes[1].imshow(color_mask)
            axes[1].set_title("Masque Colorisé")
            axes[1].axis('off')
            plt.tight_layout()
            plt.show()

            # Afficher un résumé des classes détectées
            readable_counts = { [k for k,v in CLASS_MAPPING.items() if v==cid][0]: cnt for cid, cnt in class_counts.items() }
            print("📊 Pixels par classe (top 10):")
            for cls_name, cnt in sorted(readable_counts.items(), key=lambda x: -x[1])[:10]:
                pct = cnt / mask.size * 100
                print(f" - {cls_name:<15}: {cnt:>6} px ({pct:5.2f}%)")
        else:
            print(f"❌ Erreur lors de la segmentation : {response.status_code}")
            print(f"📝 Détails : {response.text}")

    except Exception as e:
        print(f"Une erreur est survenue : {e}")
else:
    print("Aucune image à traiter. Vérifiez la configuration de 'image_dir' et 'max_images'.")

## 5. Segmentation de Plusieurs Images (Batch)

Maintenant que nous savons comment traiter une image, nous pouvons créer une fonction pour en traiter plusieurs.
Cette fonction va boucler sur la liste `image_paths` et appliquer la logique de segmentation à chaque image.
Nous utiliserons `tqdm` pour avoir une barre de progression.

In [ ]:
def segment_images_batch(list_of_image_paths):
    """
    Segmente une liste d'images en utilisant l'API Hugging Face.

    Args:
        list_of_image_paths (list): Liste des chemins vers les images.

    Returns:
        list: Liste des masques de segmentation (tableaux NumPy).
              Contient None si une image n'a pas pu être traitée.
    """
    batch_segmentations = []

    # N'oubliez pas de mettre une pause entre chaque appel API !
    for img_path in tqdm(list_of_image_paths, desc="Segmentation des images en batch"):
        try:
            with open(img_path, "rb") as image_file:
                image_data = image_file.read()

            headers["Content-Type"] = "image/png"
            response = requests.post(API_URL, headers=headers, data=image_data)

            if response.status_code == 200:
                segmentation_result = response.json()
                width, height = get_image_dimensions(img_path)
                mask = create_masks(segmentation_result, width, height, CLASS_MAPPING)
                batch_segmentations.append(mask)
            else:
                print(f"❌ Erreur lors de la segmentation de {img_path} : {response.status_code}")
                batch_segmentations.append(None)

        except Exception as e:
            print(f"Une erreur est survenue avec {img_path} : {e}")
            batch_segmentations.append(None)

        # Pause entre les appels API
        time.sleep(PAUSE_BETWEEN_CALLS)

    return batch_segmentations

# Appeler la fonction pour segmenter les images listées dans image_paths
if image_paths:
    print(f"\nTraitement de {len(image_paths)} image(s) en batch...")
    batch_seg_results = segment_images_batch(image_paths)
    print("Traitement en batch terminé.")
else:
    batch_seg_results = []
    print("Aucune image à traiter en batch.")

## 6. Affichage des Résultats en Batch

Nous allons maintenant créer une fonction pour afficher les images originales et leurs segmentations correspondantes côte à côte, dans une grille.

In [ ]:
def display_segmented_images_batch(original_image_paths, segmentation_masks):
    """
    Affiche les images originales et leurs masques segmentés.

    Args:
        original_image_paths (list): Liste des chemins des images originales.
        segmentation_masks (list): Liste des masques segmentés (NumPy arrays).
    """
    # Matplotlib, ça vous parle ?
    # Alors... au travail ! 😉

# Afficher les résultats du batch
if batch_seg_results:
    display_segmented_images_batch(image_paths, batch_seg_results)
else:
    print("Aucun résultat de segmentation à afficher.")

## Conclusion et Prochaines Étapes

Félicitations ! Vous avez appris à :
- Configurer les appels à l'API d'inférence Hugging Face.
- Envoyer des images pour la segmentation.
- Interpréter les résultats (avec l'aide des fonctions fournies).
- Visualiser les segmentations.

Pistes d'amélioration ou d'exploration :
- **Gestion d'erreurs plus fine** : Implémenter des tentatives multiples (retry) en cas d'échec de l'API (par exemple, si le modèle est en cours de chargement).
- **Appels asynchrones** : Pour un grand nombre d'images, des appels asynchrones (avec `asyncio` et `aiohttp`) seraient beaucoup plus rapides.
- **Autres modèles** : Explorer d'autres modèles de segmentation ou d'autres tâches sur Hugging Face Hub.

N'hésitez pas à modifier le code, à tester avec vos propres images et à explorer davantage !

**_Note_** : Si vous aimez ce modèle, n'hésitez pas à le [télécharger](https://huggingface.co/sayeed99/segformer_b3_clothes) et jouer avec directement sur votre machine !